In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### implementation of basic ae

In [109]:
class Evolutionary_Algorithm:
    __slots__ = [
        "mutation_rate",
        "crossover_rate",
        "number_of_generations",
        "population_size",
        "function_to_optimize",
        "population",
        "problem_dim",
        "hall_of_fame",
    ]

    def __init__(
        self,
        mutation_rate=0.7,
        crossover_rate=0.7,
        number_of_generations=50,
        population_size=100,
    ):
        self.population_size = population_size
        self.number_of_generations = number_of_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.function_to_optimize = None
        self.population = None
        self.problem_dim = None
        self.hall_of_fame = []

    def _generate_population(self, n_genes):
        self.population = np.random.uniform(-10, 10, (self.population_size, n_genes))

    def _mutation(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual += np.random.normal(0, 1, self.problem_dim)
        return individual

    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.problem_dim - 1)
            child1 = np.concatenate(
                (parent1[:crossover_point], parent2[crossover_point:])
            )
            child2 = np.concatenate(
                (parent2[:crossover_point], parent1[crossover_point:])
            )
            return child1, child2
        return parent1, parent2

    def _evaluate_population(self):
        fitness = np.zeros(len(self.population))
        for i, individual in enumerate(self.population):
            fitness[i] = self.function_to_optimize(*individual)
        self.hall_of_fame.append(np.argsort(fitness)[: int(self.population_size * 0.1)])
        return fitness

    def _tournament_selection(self, fitness):
        fitness = 1 / fitness
        probabilities = fitness / np.sum(fitness)
        return self.population[
            np.random.choice(
                range(len(self.population)),
                p=probabilities,
                size=self.population_size - len(self.hall_of_fame[-1]),
                replace=True,
            )
        ]

    def _select_individual(self):
        return self.population[np.random.choice(range(self.population_size))]

    def _visualize_individual(self, individual):
        print(individual)

    def optimize(self, function_to_optimize):
        # initialize the population
        self.function_to_optimize = function_to_optimize
        self.problem_dim = function_to_optimize.__code__.co_argcount
        self._generate_population(self.problem_dim)

        for generation in range(self.number_of_generations):

            # crossover
            for _ in range(0, self.population_size, 2):
                parent1 = self._select_individual()
                parent2 = self._select_individual()
                child1, child2 = self._crossover(parent1, parent2)
                self.population = np.vstack([self.population, child1])
                self.population = np.vstack([self.population, child2])

            # mutation
            for _ in range(self.population_size):
                individual = self._select_individual()
                self.population = np.vstack(
                    [self.population, self._mutation(individual)]
                )

            # evaluate the population and select the best individuals
            fitness = self._evaluate_population()
            print(
                f"iteration {generation} best fitness: {np.min(fitness)} for individual {self.population[self.hall_of_fame[-1][0]]}"
            )
            self.population = np.vstack(
                [self.population[self.hall_of_fame[-1]], self.population]
            )

### definition of objective functions 

In [110]:
def rastrigin(*args):
    return 10 * len(args) + np.sum(args**2 - 10 * np.cos(2 * np.pi * args))


def rastrigin_5d(x, y, z, w, v):
    return rastrigin(x, y, z, w, v)


def quadratic_3d(x, y, z):
    return x**2 + y**2 + 2 * z**2

In [113]:
ea = Evolutionary_Algorithm(number_of_generations=200, population_size=20)
ea.optimize(quadratic_3d)

iteration 0 best fitness: 25.245944823747017 for individual [3.88290083 0.70043519 2.19982005]
iteration 1 best fitness: 21.416064201296727 for individual [ 3.95566444 -0.00900624  1.69833771]
iteration 2 best fitness: 21.416064201296727 for individual [ 3.95566444 -0.00900624  1.69833771]
iteration 3 best fitness: 13.355021439575891 for individual [-3.17697563 -0.94963336  1.08628812]
iteration 4 best fitness: 2.5271173498867263 for individual [ 1.37139638 -0.45839407 -0.46704614]
iteration 5 best fitness: 0.6563911438004058 for individual [ 0.10000913 -0.45839407 -0.46704614]
iteration 6 best fitness: 0.6563911438004058 for individual [ 0.10000913 -0.45839407 -0.46704614]
iteration 7 best fitness: 0.6563911438004058 for individual [ 0.10000913 -0.45839407 -0.46704614]
iteration 8 best fitness: 0.6563911438004058 for individual [ 0.10000913 -0.45839407 -0.46704614]
iteration 9 best fitness: 0.6563911438004058 for individual [ 0.10000913 -0.45839407 -0.46704614]
iteration 10 best fitne